In [2]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
import dask.array as da
import numpy as np
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType, Avg_Parameters

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, IlluminationCorrection, \
                                Calculate_BIGFISH_Threshold, DilationedCytoMask, CellProperties

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, \
                                    remove_local_data

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

from src.Analysis import AnalysisManager

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


# Illumination Correction

In [ ]:
src_path

In [ ]:
def runner_ip(idl):
    # Initialize Parameters
    scope = ScopeClass()
    scope.voxel_size_yx = 160 # we technically dont need these 
    data = DataContainer()
    settings = Settings(name='ER_ICP', ) 
    experiment = Experiment()
    experiment.initial_data_location = idl
    data.clear_after_error = False

    settings.load_in_mask = False
    experiment.nucChannel = 1
    settings.psuedoCyto = 0
    experiment.voxel_size_z = 500

    settings.sigma_dict = [100, 100]


    FFF2NativeDataType()
    # add illumination correction
    IlluminationCorrection()
    Save_Outputs()
    return_to_NAS()
    remove_local_data()

    Parameters.validate()

    pipeline = Pipeline()

    pipeline.run()

    pipeline.clear_pipeline() # I dont think this is nesscary but its probablily good to do

initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_0min_050823',
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_0min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_0min_050823',        
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_0min_control_062923_R2',
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_0min_control_080823_R3' )


runner_ip(initial_data_location)

# run all

In [ ]:
def runner(idl, ip):
    # Initialize Parameters
    scope = ScopeClass()
    scope.voxel_size_yx = 160 # we technically dont need these 
    data = DataContainer()
    replica = ip[0]['replica']
    settings = Settings(name=f"GR_wIC_{replica}_072925")
    experiment = Experiment()
    experiment.independent_params = ip
    experiment.initial_data_location = idl
    data.clear_after_error = False

    settings.load_in_mask = True
    experiment.nucChannel = 1
    settings.psuedoCyto = 0
    experiment.voxel_size_z = 500

    settings.sigma_dict = [50, 50]
    settings.cellpose_min_size = 100
    settings.cellpose_diameter = [180, 90]
    settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei']
    # settings.imported_profiles = np.load('/home/ericron/AngelFISH/Publications/Ron_2024/illumination_profile_011725.npy').tolist()

    FFF2NativeDataType()
    # Avg_Parameters()
    IlluminationCorrection()  # add for illumination correction
    # SimpleCellposeSegmentaion()
    DilationedCytoMask()
    CellProperties()
    Save_Masks()
    Save_Images() # include if you want to save the corrected images
    Save_Parameters()
    Save_Outputs()
    return_to_NAS()
    remove_local_data()

    Parameters.validate()

    pipeline = Pipeline()
    pipeline.isolate()

    pipeline.run_on_cluster('/home/ericron/AngelFISH/cluster', name=settings.name)

    pipeline.clear_pipeline() # I dont think this is nesscary but its probablily good to do
    

##  ```GR_timesweep```  experiments


In [ ]:
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_0min_050823',
        'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_10min_050823',
        'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_20min_050823',
        'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_30min_050823',
        'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_40min_050823',
        'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_50min_050823',
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_60min_050823',
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_75min_050823',
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_90min_050823',
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_120min_050823',
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_150min_050823',
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_180min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_0min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_10min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_20min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_30min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_40min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_50min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_60min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_75min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_90min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_120min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_150min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_180min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_0min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_10min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_20min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_30min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_40min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_50min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_60min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_75min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_90min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_120min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_150min_050823' ,
        'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_180min_050823' )
# Parameters
Condition = 'GR_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' 
Replica = 'A'                   # Replica name. 
time_list  = [0, 10, 20, 30, 40, 50, 60, 75, 90, 120, 150, 180, 0, 10, 20, 30, 40, 50, 60, 75, 90, 120, 150, 180, 0, 10, 20, 30, 40, 50, 60, 75, 90, 120, 150, 180]        # Time of image acquisition
DexConc_list = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]        # Dex concentration


# Creating the list of dictionaries
GR_ICC_R1 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]

# Ensuring the length matches the number of directories
assert len(GR_ICC_R1) == len(initial_data_location), "Mismatch between number of directories and experimental parameters."

# Output the result
print(GR_ICC_R1)

In [ ]:
runner(initial_data_location, GR_ICC_R1)

In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_0min_control_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_1nM_10min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_1nM_30min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_1nM_50min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_1nM_75min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_1nM_120min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230707_GR_ICC_R2/GR_ICC_1nM_180min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_10nM_10min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_10nM_30min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_10nM_50min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_10nM_75min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_10nM_120min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_10nM_180min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_100nM_10min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_100nM_30min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_100nM_50min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_100nM_75min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_100nM_120min_062923_R2' ,
        'smFISH_images/Eric_smFISH_images/20230713_GR_ICC_R2/GR_ICC_100nM_180min_062923_R2' )
# Parameters
Condition = 'GR_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'B'                   # Replica name. 
time_list  = [0, 10, 30, 50, 75, 120, 180, 10, 30, 50, 75, 120, 180, 10, 30, 50, 75, 120, 180]        # Time of image acquisition
DexConc_list = [0, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10, 100, 100, 100, 100, 100, 100]        # Dex concentration

# Creating the list of dictionaries
GR_ICC_R2 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(GR_ICC_R2)

In [ ]:
runner(initial_data_location, GR_ICC_R2)

In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_0min_control_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_1nM_10min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_1nM_30min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_1nM_50min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_1nM_75min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_1nM_120min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_1nM_180min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_10nM_10min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_10nM_30min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_10nM_50min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230809_GR_ICC_R3/GR_ICC_Dex_3hr_10nM_75min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_10nM_120min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_10nM_180min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_100nM_10min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_100nM_30min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_100nM_50min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_100nM_75min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_100nM_120min_080823_R3' ,
        'smFISH_images/Eric_smFISH_images/20230823/GR_ICC_Dex_3hr_100nM_180min_080823_R3' )
# Parameters
Condition = 'GR_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' 
Replica = 'C'                   # Replica name. 
time_list  = [0, 10, 30, 50, 75, 120, 180, 10, 30, 50, 75, 120, 180, 10, 30, 50, 75, 120, 180]        # Time of image acquisition
DexConc_list = [0, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10, 100, 100, 100, 100, 100, 100]        # Dex concentration

# Creating the list of dictionaries
GR_ICC_R3 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(GR_ICC_R3)

In [ ]:
runner(initial_data_location, GR_ICC_R3)